In [2]:
fn = 'capture.pcap'

In [3]:
import binascii
import dpkt
import io
import struct
import sys
from collections import Counter
from ipaddress import ip_address

In [4]:
def walk_pcap(f):
    pcap = dpkt.pcap.Reader(f)
    dlt = pcap.datalink()

    for t, data in pcap:
        try:
            if dlt == 0:
                x = struct.unpack('I', data[:4])[0]
                if x == 2:
                    ip = dpkt.ip.IP(data[4:])
                elif x == 30:
                    ip = dpkt.ip6.IP6(data[4:])
                else:
                    continue

            elif dlt == 1:
                eth = dpkt.ethernet.Ethernet(data)
                ip = eth.data
                if isinstance(ip, bytes):
                    ip = dpkt.ip.IP(ip)

            elif dlt == 101:
                ip = dpkt.ip.IP(data)

            else:
                trace('** skipping dlt', dlt)
                continue

            yield (t, eth)

        except GeneratorExit:
            raise
        except:
            # if VERBOSE: trace(sys.exc_info())
            pass

In [6]:
stats = Counter()
with open(fn, 'rb') as fp:
    for t, eth in walk_pcap(fp):
        stats[type(eth.data)] += 1
        if isinstance(eth.data, (dpkt.ip.IP, dpkt.ip6.IP6)):
            stats[type(eth.data.data)] += 1
        else:
            print(eth)
stats

Counter({dpkt.ip.IP: 39, dpkt.tcp.TCP: 39})

In [11]:
stats = Counter()
with open(fn, 'rb') as fp:
    for t, eth in walk_pcap(fp):
        if isinstance(eth.data, (dpkt.ip.IP, dpkt.ip6.IP6)):
            ip = eth.data
            if isinstance(ip.data, (dpkt.tcp.TCP, dpkt.udp.UDP)):
                tcp = ip.data
                stats[tcp.dport] += 1
stats

Counter({443: 24, 57567: 6, 57578: 4, 57581: 5})

In [19]:
ses = dict()
with open(fn, 'rb') as fp:
    for t, eth in walk_pcap(fp):
        ip = eth.data
        tcp = ip.data
        p = tcp.dport if tcp.sport == 443 else tcp.sport
        s = ses.get(p, list())
        s.append(ip)
        ses[p] = s
ses.keys()

dict_keys([57567, 57578, 57581])

In [31]:
[*map(len, ses.values())]

[16, 10, 13]

In [29]:
for a, b in zip(ses[57567], ses[57581]):
    x, y = a.data.data, b.data.data
    print(x == y)

True
True
True
False
True
False
True
False
False
True
False
False
True
